2021년 9월 27일까지의 데이터로 학습시킴

#필요 라이브러리

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install finance-datareader

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

import os
import FinanceDataReader as fdr
import datetime as dt
from datetime import datetime

In [ ]:
learning_rate = 1000
dropout_rate = 0.2
l2_rate = 0.01
minibatch_size=64 #원래 128이었는데 메모리 부족으로 fine tuning에서 16로 내림
num_epochs_pretraining = 1
num_epochs = 200

In [ ]:
first_year = '2015' # '2011'은 pretraining 때
start_date = '0101' + str(int(first_year)%100).zfill(2)
end_date = datetime.today().strftime("%m%d%y")

days = 90  # days를 바꾸면 전체 모델을 바꿔야 하므로 주의!!! 특히 model2
sample_num = 100

#데이터셋 불러오기

In [ ]:
path = '/content/drive/MyDrive/stock/data/'
list_name = 'Stock_List.csv'
stock_list = pd.read_csv(os.path.join(path, list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list

,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI
...,...,...,...
371,더네이쳐홀딩스,298540,KOSDAQ
372,코엔텍,029960,KOSDAQ
373,원익홀딩스,030530,KOSDAQ
374,웹케시,053580,KOSDAQ


In [ ]:
# 데이터에 etf가 있기는 하다....
kospi_codes = []
kospi_coms = {}
kosdaq_codes = []
kosdaq_coms = {}

In [ ]:
for i in range(len(stock_list)):
    code = stock_list['종목코드'][i]
    if (stock_list['상장시장'][i] == 'KOSPI'):
        kospi_codes.append(code)
        kospi_coms[code] = stock_list['종목명'][i]
    else:
        kosdaq_codes.append(code)
        kosdaq_coms[code] = stock_list['종목명'][i]

In [ ]:
codes = {'kospi' : kospi_codes, 'kosdaq' : kosdaq_codes}

In [ ]:
from datetime import datetime

def get_prices(code = '005930', start_date = '010115', end_date = datetime.today().strftime("%m%d%y")):
    # 한국거래소 주식 시장은 1월 1일과 12월 31일, 공휴일, 주말이 휴장이다.
    stock = fdr.DataReader(code, start = start_date, end = end_date) # 날짜: 월 일 연도
    return stock

# 데이터셋 전처리

In [ ]:
def add_liquidity(prices):
    # 유동성 특성 추가
    prices.loc[:, 'O-C'] = prices['Open'] - prices['Close']
    prices.loc[:, 'H-L'] = prices['High'] - prices['Low']
    prices.loc[:, 'O-L'] = prices['Open'] - prices['Low']
    prices.loc[:, 'O-H'] = prices['Open'] - prices['High']
    prices.loc[:, 'C-L'] = prices['Close'] - prices['Low']
    prices.loc[:, 'C-H'] = prices['Close'] - prices['High']
    return prices

In [ ]:
def rename_col(data, name):
    for col in data.columns:
        data = data.rename({col : name + '_' + col}, axis=1)
    return data

마켓 데이터 불러오기

In [ ]:
class market_data():
    def __init__(self, first_year='2015'):
        # 한국거래소, first_year~현재
        ks11 = fdr.DataReader('KS11', first_year) #코스피 지수
        kq11 = fdr.DataReader('KQ11', first_year) #코스닥 지수
        ks50 = fdr.DataReader('KS50', first_year) #코스피50 지수
        ks100 = fdr.DataReader('KS100', first_year) #코스피100
        ks200 = fdr.DataReader('KS200', first_year) #코스피200
        krx100 = fdr.DataReader('KRX100', first_year) #KRX 100

        # 미국 지수, first_year~현재
        dji = fdr.DataReader('DJI', first_year) #다우존스 지수
        ixic = fdr.DataReader('IXIC', first_year) #나스닥 지수
        us500 = fdr.DataReader('US500', first_year) #S&P500 지수
        vix = fdr.DataReader('VIX', first_year) #S&P VIX

        # 원달러 환율, first_year~현재
        usd_krw = fdr.DataReader('USD/KRW', first_year)

        # 유동성 특성 추가
        ks11 = add_liquidity(ks11)
        kq11 = add_liquidity(kq11)
        ks50 = add_liquidity(ks50)
        ks100 = add_liquidity(ks100)
        ks200 = add_liquidity(ks200)
        krx100 = add_liquidity(krx100)

        usd_krw = add_liquidity(usd_krw)

        dji = add_liquidity(dji)
        ixic = add_liquidity(ixic)
        us500 = add_liquidity(us500)
        vix = add_liquidity(vix)

        # 열 이름 바꾸기
        self.ks11 = rename_col(ks11, 'KS11')
        self.kq11 = rename_col(kq11, 'KQ11')
        self.ks50 = rename_col(ks50, 'KS50')
        self.ks100 = rename_col(ks100, 'KS100')
        self.ks200 = rename_col(ks200, 'KS200')
        self.krx100 = rename_col(krx100, 'KRX100')

        self.dji = rename_col(dji, 'DJI')
        self.ixic = rename_col(ixic, 'IXIC')
        self.us500 = rename_col(us500, 'US500')
        self.vix = rename_col(vix, 'VIX')

        self.usd_krw = rename_col(usd_krw, 'USD/KRW')

    def get_com_data(self, code = '005930', start_date = '010115', end_date = datetime.today().strftime("%m%d%y")):
        
        prices = get_prices(code = code, start_date = start_date, end_date = end_date)
        prices = add_liquidity(prices)
        
        # 해당 기업, 코스피 지수, 코스닥 지수, 코스피50 지수, 코스피100 지수, 코스피200 지수, KRX100 지수, 달러원 환율, 다우 지수, 나스닥 지수, S&P500 지수, VIX 지수
        com_data = pd.concat([prices, self.ks11, self.kq11, self.ks50, self.ks100, self.ks200, self.krx100, self.usd_krw, self.dji, self.ixic, self.us500, self.vix], axis=1)
       
       # 결측치는 모두 없애기...다른 방법 찾아보자
        com_data = com_data.dropna(axis=0)

        return com_data

market_data = market_data(first_year=first_year)

In [ ]:
sample_com = market_data.get_com_data(code = '005930', start_date=start_date, end_date=end_date)
print(sample_com.shape)
sample_com.tail()

(1608, 143)


,Open,High,Low,Close,Volume,Change,O-C,H-L,O-L,O-H,C-L,C-H,KS11_Close,KS11_Open,KS11_High,KS11_Low,KS11_Volume,KS11_Change,KS11_O-C,KS11_H-L,KS11_O-L,KS11_O-H,KS11_C-L,KS11_C-H,KQ11_Close,KQ11_Open,KQ11_High,KQ11_Low,KQ11_Volume,KQ11_Change,KQ11_O-C,KQ11_H-L,KQ11_O-L,KQ11_O-H,KQ11_C-L,KQ11_C-H,KS50_Close,KS50_Open,KS50_High,KS50_Low,...,DJI_O-L,DJI_O-H,DJI_C-L,DJI_C-H,IXIC_Close,IXIC_Open,IXIC_High,IXIC_Low,IXIC_Volume,IXIC_Change,IXIC_O-C,IXIC_H-L,IXIC_O-L,IXIC_O-H,IXIC_C-L,IXIC_C-H,US500_Close,US500_Open,US500_High,US500_Low,US500_Volume,US500_Change,US500_O-C,US500_H-L,US500_O-L,US500_O-H,US500_C-L,US500_C-H,VIX_Close,VIX_Open,VIX_High,VIX_Low,VIX_Volume,VIX_Change,VIX_O-C,VIX_H-L,VIX_O-L,VIX_O-H,VIX_C-L,VIX_C-H
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-09-17,76300.0,77200.0,75900.0,77200.0,16289502.0,0.014455,-900.0,1300.0,400.0,-900.0,1300.0,0.0,3140.51,3127.75,3142.13,3112.51,589240000.0,0.0033,-12.76,29.62,15.24,-14.38,28.00,-1.62,1046.12,1040.73,1046.13,1033.69,1.160000e+09,0.0064,-5.39,12.44,7.04,-5.40,12.43,-0.01,2826.56,2810.05,2828.55,2798.64,...,218.23,-42.00,65.25,-194.98,15044.0,15163.4,15166.6,14998.7,2.750000e+09,-0.0091,119.4,167.9,164.7,-3.2,45.3,-122.6,4432.99,4469.74,4471.52,4427.76,0.0,-0.0091,36.75,43.76,41.98,-1.78,5.23,-38.53,20.81,18.37,21.51,18.35,0.0,0.1134,-2.44,3.16,0.02,-3.14,2.46,-0.70
2021-09-23,77500.0,77600.0,76800.0,77400.0,17055590.0,0.002591,100.0,800.0,700.0,-100.0,600.0,-200.0,3127.58,3123.64,3131.63,3107.98,623840000.0,-0.0041,-3.94,23.65,15.66,-7.99,19.60,-4.05,1036.26,1040.38,1043.58,1036.25,9.681700e+08,-0.0094,4.12,7.33,4.13,-3.20,0.01,-7.32,2825.27,2814.16,2827.21,2805.06,...,0.00,-583.48,469.70,-113.78,15052.2,14960.0,15085.4,14932.5,7.882300e+08,0.0104,-92.2,152.9,27.5,-125.4,119.7,-33.2,4448.98,4406.75,4465.40,4406.75,0.0,0.0121,-42.23,58.65,0.00,-58.65,42.23,-16.42,18.63,19.91,20.21,18.42,0.0,-0.1073,1.28,1.79,1.49,-0.30,0.21,-1.58
2021-09-24,77600.0,77700.0,77100.0,77300.0,12247274.0,-0.001292,300.0,600.0,500.0,-100.0,200.0,-400.0,3125.24,3140.73,3146.86,3119.40,753220000.0,-0.0007,15.49,27.46,21.33,-6.13,5.84,-21.62,1037.03,1040.43,1045.21,1034.80,1.310000e+09,0.0007,3.40,10.41,5.63,-4.78,2.23,-8.18,2824.39,2836.02,2841.23,2819.50,...,78.79,-127.45,146.92,-59.32,15047.7,14961.6,15067.6,14946.6,7.734400e+08,-0.0003,-86.1,121.0,15.0,-106.0,101.1,-19.9,4455.48,4438.04,4463.12,4430.27,0.0,0.0015,-17.44,32.85,7.77,-25.08,25.21,-7.64,17.75,19.33,20.41,17.63,0.0,-0.0472,1.58,2.78,1.70,-1.08,0.12,-2.66
2021-09-27,77300.0,77700.0,77000.0,77700.0,11699798.0,0.005175,-400.0,700.0,300.0,-400.0,700.0,0.0,3133.64,3121.70,3146.35,3119.26,883230000.0,0.0027,-11.94,27.09,2.44,-24.65,14.38,-12.71,1034.82,1037.21,1039.63,1031.93,1.050000e+09,-0.0021,2.39,7.70,5.28,-2.42,2.89,-4.81,2831.18,2820.70,2842.57,2818.39,...,0.00,-250.48,60.39,-190.09,14970.0,14954.2,15003.2,14865.0,8.946100e+08,-0.0052,-15.8,138.2,89.2,-49.0,105.0,-33.2,4443.11,4442.12,4457.30,4436.19,0.0,-0.0028,-0.99,21.11,5.93,-15.18,6.92,-14.19,18.76,17.78,19.32,17.74,0.0,0.0569,-0.98,1.58,0.04,-1.54,1.02,-0.56
2021-09-28,77700.0,77800.0,76200.0,76300.0,15406382.0,-0.018018,1400.0,1600.0,1500.0,-100.0,100.0,-1500.0,3097.92,3133.40,3134.46,3095.72,881960.0,-0.0114,35.48,38.74,37.68,-1.06,2.20,-36.54,1012.51,1034.04,1034.07,1012.51,1.350000e+06,-0.0216,21.53,21.56,21.53,-0.03,0.00,-21.56,2795.88,2829.37,2831.23,2792.84,...,493.05,-47.56,44.88,-495.73,14546.7,14787.2,14817.7,14539.9,9.962600e+08,-0.0283,240.5,277.8,247.3,-30.5,6.8,-271.0,4352.63,4419.54,4419.54,4346.33,0.0,-0.0204,66.91,73.21,73.21,0.00,6.30,-66.91,23.25,18.76,24.82,18.76,0.0,0.2393,-4.49,6.06,0.00,-6.06,4.49,-1.57


In [ ]:
sample_com.index[-1] + dt.timedelta(days=1)

Timestamp('2021-09-29 00:00:00')

마켓 데이터에서 랜덤 샘플링

In [ ]:
def get_training_data(sample_num, code = '005930', days=365, start_date = '010115', end_date = datetime.today().strftime("%m%d%y")):
    com = market_data.get_com_data(code = code, start_date = start_date, end_date = end_date)
    last_data = com.iloc[-1]
    dataset = com.iloc[: -1]

    X = np.zeros((sample_num, days, 143))
    y = np.zeros((sample_num, 1, 1))

    # 데이터 랜덤 샘플링
    for i in range(sample_num-1):
        replace = False
        if dataset.shape[0] < days*2:
            replace = True
        # 전체 데이터에서 days개 랜덤 샘플링
        # 상장된지 얼마 안되면 중복 추출 허용
        train_set = dataset.sample(days, replace=replace, random_state=21).sort_index()

        # 랜덤 샘플링 데이터의 마지막 데이터의 다음 날 데이터
        target_day = train_set.index[-1] + dt.timedelta(days=1)
        if target_day == com.index[-1]:
            target = last_data
        else:
            while 1:
              if target_day in com.index:
                  target = com.loc[target_day]
                  break
              else:
                  target_day = target_day + dt.timedelta(days=1)
        target = target['Close']

        X[i, :, :] = train_set.to_numpy()
        y[i, :, :] = np.array(target)

    # 최근 1년 데이터 샘플링
    train_set = com.iloc[-days-1: -1] 
    target = last_data
    target = target['Close']

    X[sample_num-1, :, :] = train_set.to_numpy()
    y[sample_num-1, :, :] = np.array(target)

    return X, y

데이터프레임 자료이므로 .to_numpy 해야 한다.



#모델

먼저 기업을 classification을 하고 그 class에 맞는 주가 예측 모델을 통과하면?
1. 기업을 classification
2. class에 맞는 기업에 대한 regeression

시계열 모델를 사용하여 다음에 올 단어(다음 날 주가)를 맞추는 방식으로 첫 몇 년은 그냥 트레이닝 데이터로 사용하고 다음 날의 주가 실험하고 날을 거듭할 수록 누적. 오늘이 된 다음날의 데이터를 새로운 트레이닝 데이터로 적용.

In [ ]:
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, GRU, Activation, Dropout
from tensorflow.keras.regularizers import L2

from tensorflow.keras.initializers import GlorotNormal
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanAbsoluteError

from sklearn.model_selection import train_test_split

In [ ]:
tf.random.set_seed(21)
np.random.seed(21)

##1. 하루 단위 피쳐 학습

In [ ]:
class Model1(tf.keras.Model):
    def __init__(self, dropout_rate=0.2, l2_rate=0.01):
        super(Model1, self).__init__()

        self.fc1 = Dense(143, activation='relu', kernel_regularizer=L2(l2_rate))
        self.dropout1 = Dropout(dropout_rate)

        self.fc2 = Dense(64, activation='relu', kernel_regularizer=L2(l2_rate))
        self.dropout2 = Dropout(dropout_rate)
        
        self.ffn = Dense(12, activation='relu', kernel_regularizer=L2(l2_rate))
    
    def call(self, input):
                
        y1 = self.fc1(input)
        y1 = self.dropout1(y1)

        y2 = self.fc2(y1)
        y2 = self.dropout2(y2)

        y3 = self.ffn(y2)

        return y3

##2. 이를 통해 내일의 주가 예측

In [ ]:
class Model2(tf.keras.Model):
    def __init__(self, days=365, l2_rate=0.01):
        super(Model2, self).__init__()

        self.gru1 = GRU(days, activation='tanh', kernel_regularizer=L2(l2_rate), return_sequences=True) # input_shape = (12, 1),

        #self.gru2 = GRU(days, activation='tanh', kernel_regularizer=L2(l2_rate), return_sequences=True) # 메모리 부족으로

        self.gru3 = GRU(days, activation='tanh', kernel_regularizer=L2(l2_rate), return_sequences=False)
        
        self.ffn = Dense(1)
            
    def call(self, input):
        
        y1 = self.gru1(input)

        #y2 = self.gru2(y1)

        y3 = self.gru3(y1)
                
        y4 = self.ffn(y3)
        
        return y4

## FULL 모델

- model1 = 1 : 1
- model2 = X_train.shape[0] : 1
<br>비율이 맞지 않아 model1을 X_train.shape[0] 다 하고 model2로 넘어감 하지만 gradient descent 작업은 한 번에 하기 위해 tf.Gradient Tape 사용

In [ ]:
initialize_parameters = GlorotNormal()
optimizer = Adam(learning_rate)
compute_loss = MeanAbsoluteError()

In [ ]:
class fullModel(tf.keras.Model):
    def __init__(self, code='005930', days=365, dropout_rate=0.2, l2_rate=0.01):
        super(fullModel, self).__init__()
        
        self.modelFC = Model1(dropout_rate=dropout_rate, l2_rate=l2_rate)

        self.modelGRU = Model2(days=days, l2_rate=l2_rate)

        self.modelFC.compile(loss='mean_absolute_error', optimizer=optimizer, metrics=['mean_absolute_error'])
        self.modelGRU.compile(loss='mean_absolute_error', optimizer=optimizer, metrics=['mean_absolute_error'])

        # weights load
        path ='/content/drive/MyDrive/stock/weights/' + code
        self.path1 = path + '/model01/weights'
        self.path2 = path + '/model02/weights'
        if os.path.isdir(path):
            self.modelFC.load_weights(self.path1)
            self.modelGRU.load_weights(self.path2)
        elif os.path.isdir('/content/drive/MyDrive/stock/weights/005930'):
            print('~loading pretrained~', end=' ')
            self.modelFC.load_weights('/content/drive/MyDrive/stock/weights/005930/model01/weights')
            self.modelGRU.load_weights('/content/drive/MyDrive/stock/weights/005930/model02/weights')
    
    def save_weights(self):
        self.modelFC.save_weights(self.path1)
        self.modelGRU.save_weights(self.path2)
    
    def call(self, input):
        y1 = self.modelFC(input)

        if y1.ndim == 2:
            y1 = y1[np.newaxis, :, :]
        
        y_hat = self.modelGRU(y1)

        return y_hat

## 한 기업 한 구간

In [ ]:
def compute_cost(logits, labels):

    cost = tf.reduce_mean(compute_loss(y_pred=logits, y_true=labels))
    
    return cost

In [ ]:
def train_com_one_step(X_train, y_train, code='005930', days=365, learning_rate = 0.01, dropout_rate=0.2, l2_rate=0.01):
    tf.keras.backend.clear_session()
  
    optimizer = Adam(learning_rate)

    Model = fullModel(code=code, days=days, dropout_rate=dropout_rate, l2_rate=l2_rate)

    with tf.GradientTape() as tape:
        # 1. predict
        y_hat = Model(X_train)

        # 2. loss
        cost = compute_cost(logits=y_hat, labels=y_train)

    grads = tape.gradient(cost, Model.trainable_variables)
    optimizer.apply_gradients(zip(grads, Model.trainable_variables))

    # weights save
    Model.save_weights()

    return cost

전체 모델은 앞서 model1에서 model2로 넘어가는 FullModel 유닛들은 스태킹하기.<br>
유닛들은 1차로 해당 기업과 코스피, 코스닥, 달러원환율, 다우, 나스닥, S&P, VIX로 총 8개 유닛의 스태킹.<br>
2차는 모든 기업을 포함시킨 스태킹(가능하다면)

##한 기업 전체 구간

In [ ]:
def train_com(sample_num=100, code = '005930', days=365, start_date = '010115', end_date = datetime.today().strftime("%m%d%y"),
                 learning_rate = 0.01, dropout_rate=0.2, l2_rate=0.01, minibatch_size=12, num_epochs=100, print_cost=True):

    X, y = get_training_data(sample_num=sample_num, code=code, days=days, start_date=start_date, end_date=end_date)

    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=21)

    #y_train = y_train.reshape((-1, 1))
    #y_valid = y_valid.reshape((-1, 1))

    #numpy를 tensorflow dataset으로
    train_set = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    valid_set = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))

    #train_set = tf.data.Dataset.zip((X_train, y_train))
    #valid_set = tf.data.Dataset.zip((X_valid, y_valid))

    m = train_set.cardinality().numpy()
    n = valid_set.cardinality().numpy()

    train_minibatches = train_set.batch(minibatch_size).prefetch(8)
    valid_minibatches = valid_set.batch(minibatch_size).prefetch(8)

    train_costs = []
    valid_costs = []

    for epoch in range(1, num_epochs+1):
              
        #if epoch % 100 == 0:
        #    print(" Epoch", epoch, end=' ')

        epoch_cost = 0.

        for (minibatch_X, minibatch_y) in train_minibatches:
            minibatch_cost = train_com_one_step(minibatch_X, minibatch_y, code=code, learning_rate = learning_rate, dropout_rate=dropout_rate, l2_rate=l2_rate)
            epoch_cost += minibatch_cost

        epoch_cost /= m

        if epoch % 10 == 0:
            print('>', end=' ')
        elif epoch % 5 == 0:
            print('\'', end='')
        else:
            print(".", end='')

        if epoch % 10 == 0:
                        
            valid_cost = 0.         
            for (minibatch_X, minibatch_y) in valid_minibatches:
                minibatch_cost = train_com_one_step(minibatch_X, minibatch_y, code=code, learning_rate = learning_rate, dropout_rate=dropout_rate, l2_rate=l2_rate)
                valid_cost += minibatch_cost
            valid_cost /= n

            train_costs.append(epoch_cost)
            valid_costs.append(valid_cost)
            
            if print_cost == True:
                print("After epoch %i : Train Cost %f" % (epoch, epoch_cost), end=" ")
                print("Valid Cost %f" % (valid_cost))
            
            if epoch_cost < 1 and valid_cost < 1 and epoch_cost - valid_cost < 1: #조기 종료 조건
                break;
            
            # exploding gradient or gradinet in loop
            if (epoch >= 20 and epoch_cost >= train_costs[-2]) or (epoch > 30 and (epoch_cost == train_costs[-3])):
                learning_rate /= 10
                print('Down Learning Rate', end='')

    return train_costs, valid_costs

##Pretraining using code = '005930' 삼성전자

In [ ]:
#Model = fullModel()
#Model.build(inpt_shape=(sample_num, days, 143))
#Model.summary()

In [ ]:
train_costs, valid_costs = train_com(sample_num=sample_num, code='005930', days=days, start_date = start_date, end_date = end_date,
                                             learning_rate = learning_rate, dropout_rate=dropout_rate, l2_rate=l2_rate, minibatch_size=minibatch_size, num_epochs=num_epochs, print_cost=True)
print('Train Cost ', train_costs[-1])
print('Valid Cost ', valid_costs[-1])

....'....> After epoch 10 : Train Cost 1007.523804 Valid Cost 4248.678223
....'....> After epoch 20 : Train Cost 116.166580 Valid Cost 237.569534
....'....> After epoch 30 : Train Cost 21.356285 Valid Cost 46.564842
....'....> After epoch 40 : Train Cost 21.517969 Valid Cost 144.440628
Down Learning Rate....'....> After epoch 50 : Train Cost 2.388409 Valid Cost 8.363281
....'....> After epoch 60 : Train Cost 2.417448 Valid Cost 10.737500
Down Learning Rate....'....> After epoch 70 : Train Cost 0.504785 Valid Cost 0.721875
Train Cost  tf.Tensor(0.5047852, shape=(), dtype=float32)
Valid Cost  tf.Tensor(0.721875, shape=(), dtype=float32)


code = '005930'의 모델로 pretrained model를 만들고 다른 기업에 대해서는 fine tuning 하자

## Full training & Fine Tuning

In [ ]:
# 287410 포함 이후부터 해야 함

In [ ]:
def fullTraining(codes, sample_num=100, days=365, start_date = '010115', end_date = datetime.today().strftime("%m%d%y"),
                 learning_rate = 0.01, dropout_rate=0.2, l2_rate=0.01, minibatch_size=12, num_epochs=100, print_cost=False):
    kospi = codes['kospi']
    kosdaq = codes['kosdaq']

    for code in kospi:
        if code == '005930':
            continue
        
        print('code %s' % (code), end=' ')
        train_costs, valid_costs = train_com(sample_num=sample_num, code=code, days=days, start_date = start_date, end_date = end_date,
                                             learning_rate = learning_rate, dropout_rate=dropout_rate, l2_rate=l2_rate, minibatch_size=minibatch_size, num_epochs=num_epochs, print_cost=print_cost)
        print('Train Cost ', train_costs[-1])
        print('Valid Cost ', valid_costs[-1])
    
    for code in kosdaq:

        print('code %s' % (code), end=' ')
        train_costs, valid_costs = train_com(sample_num=sample_num, code=code, days=days, start_date = start_date, end_date = end_date,
                                             learning_rate = learning_rate, dropout_rate=dropout_rate, l2_rate=l2_rate, minibatch_size=minibatch_size, num_epochs=num_epochs, print_cost=print_cost)
        print('Train Cost ', train_costs[-1])
        print('Valid Cost ', valid_costs[-1])

In [ ]:
fullTraining(codes, sample_num=sample_num, days=days, start_date = start_date, end_date = end_date,
                 learning_rate=learning_rate, dropout_rate=dropout_rate, l2_rate=l2_rate, minibatch_size=minibatch_size, num_epochs=num_epochs, print_cost=True)

code 287410 ....'....> After epoch 10 : Train Cost 2033.514282 Valid Cost 6739.982910
....'....> After epoch 20 : Train Cost 2033.260742 Valid Cost 11560.503906
....'....> After epoch 30 : Train Cost 2033.514282 Valid Cost 6739.982910
Down Learning Rate....'....> After epoch 40 : Train Cost 203.212021 Valid Cost 580.212524
....'....> After epoch 50 : Train Cost 203.465500 Valid Cost 1249.836304
Down Learning Rate....'....> After epoch 60 : Train Cost 20.460617 Valid Cost 151.807129
....'....> After epoch 70 : Train Cost 20.352289 Valid Cost 31.197803
....'....> After epoch 80 : Train Cost 20.460617 Valid Cost 151.807129
Down Learning Rate....'....> After epoch 90 : Train Cost 2.160118 Valid Cost 5.403076
....'....> After epoch 100 : Train Cost 2.115336 Valid Cost 12.897364
....'....> After epoch 110 : Train Cost 2.160118 Valid Cost 5.403076
Down Learning Rate....'....> After epoch 120 : Train Cost 0.330078 Valid Cost 1.742969
....'....> After epoch 130 : Train Cost 0.329776 Valid Cost 

모든 기업의 예측을 토대로 스태킹

#Prediction

In [ ]:
Model = fullModel()
y_hat = Model(sample_com.to_numpy())
print(y_hat.shape)
print(y_hat)

tf.Tensor([[False]], shape=(1, 1), dtype=bool)
tf.Tensor([[False]], shape=(1, 1), dtype=bool)
tf.Tensor([[False]], shape=(1, 1), dtype=bool)
tf.Tensor([[False]], shape=(1, 1), dtype=bool)
tf.Tensor([[ True]], shape=(1, 1), dtype=bool)
tf.Tensor([[False]], shape=(1, 1), dtype=bool)
tf.Tensor([[False]], shape=(1, 1), dtype=bool)
(1, 1)
tf.Tensor([[75200.]], shape=(1, 1), dtype=float32)


In [ ]:
path = '/content/drive/MyDrive/stock/data/'
list_name = 'sample_submission_week4.csv'
stock_list = pd.read_csv(os.path.join(path, list_name))
stock_list

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,000670,000720,000810,000880,000990,001230,001440,001450,001740,002380,002790,003000,003090,003380,003410,003490,003670,003800,004000,004020,004170,004370,004490,004800,004990,005250,005290,005300,005380,005385,...,273130,278280,278530,282330,285130,287410,290510,290650,292150,293490,293780,294090,294870,298000,298020,298050,298380,298540,299030,299660,299900,307950,314130,316140,319400,319660,321550,323990,326030,330590,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-09-06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2021-09-07,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2021-09-08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2021-09-09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2021-09-10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,2021-09-27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,2021-09-28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,2021-09-29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,2021-09-30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,2021-10-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
stock_list.columns

Index(['Day', '000060', '000080', '000100', '000120', '000150', '000240',
       '000250', '000270', '000660',
       ...
       '330860', '336260', '336370', '347860', '348150', '348210', '352820',
       '357780', '363280', '950130'],
      dtype='object', length=377)

In [ ]:
m = tf.keras.metrics.Accuracy()
pred = np.array([[1], [2], [3], [4]])
true = np.array([[0], [2], [3], [4]])
m.update_state(true, pred)
m.result().numpy()

In [ ]:
true[0]